In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
import os

In [3]:
import pickle

In [4]:
import pylangacq as pla

In [5]:
import random

Load Targets

In [53]:
# y_cookie = torch.load('cookie_data_targets.pt')

In [6]:
y_cookie = torch.load('cookie_target_dict.pt')

In [7]:
len(y_cookie)

552

In [56]:
# y_sentence = torch.load('sentence_data_targets.pt')

In [8]:
y_sentence = torch.load('sentence_target_dict.pt')

In [9]:
len(y_sentence)

240

Load Samples

In [10]:
X_cookie_filenames = torch.load('cookie_fnames_dict.pt')

In [11]:
len(X_cookie_filenames)

552

In [12]:
X_sent_filenames = torch.load('sentence_fnames_dict.pt')

In [13]:
len(X_sent_filenames)

240

In [14]:
num_samples = len(X_cookie_filenames) + len(X_sent_filenames)

In [15]:
num_samples

792

In [16]:
X_cookie_reader = pla.read_chat('./Pitt/*/cookie/*.cha')
X_sentence_reader = pla.read_chat('./Pitt/*/sentence/*.cha')

-----------------------------------

Missing data -- targets

group missing or input error st no mean to replace with and left untouched in target generation

replace with closest analog as per means generated from target generation:

cookie {'Control': 30, 'ProbableAD': 19, 'MCI': 28, 'Memory': 30, 'Vascular': 17, 'PossibleAD': 20, 'Probable': 19, 'Other': 24}

sentence {'ProbableAD': 19, 'MCI': 28, 'Memory': 30, 'Vascular': 17, 'PossibleAD': 21, 'Control': 30, 'Probable': 19, 'Other': 24}

In [137]:
for sent_k in y_sentence:
    if type(y_sentence[sent_k]) == str:
        print(X_sent_filenames[sent_k])
        print(sent_k,y_sentence[sent_k])

/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/236-0.cha
674 Dementia


In [142]:
target_674 = X_sentence_retder.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/236-0.cha']['Participants']['PAR']['group']

In [143]:
target_674

''

In [144]:
group_674

'Dementia'

In [154]:
# Replace Dementia with Possible mean (as per diagnostic codes in Pitt-data.xlxs and accompanying Pitt-readme.pdf)
y_sentence[674] = 21

In [139]:
for coo_kie in y_cookie:
    if type(y_cookie[coo_kie]) == str:
        print(X_cookie_filenames[coo_kie])
        print(coo_kie,y_cookie[coo_kie])

/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha
219 
/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha
511 possibleAD


In [146]:
target_219 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha']['Participants']['PAR']['education']
group_219 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha']['Participants']['PAR']['group']

In [147]:
target_219

''

In [148]:
group_219

''

In [156]:
# Replace '' with Control mean (as per folder and dx codes in .xlxs)
y_cookie[219] = 30

In [149]:
target_511 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha']['Participants']['PAR']['education']
group_511 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha']['Participants']['PAR']['group']

In [150]:
target_511

''

In [151]:
group_511

'possibleAD'

In [158]:
# Replace possibleAD with PossibleAD mean
y_cookie[511] = 20

In [159]:
# confirm all targets filled now 
for sent_k in y_sentence:
    if type(y_sentence[sent_k]) == str:
        print(X_sent_filenames[sent_k])
        print(sent_k,y_sentence[sent_k])


In [160]:
for coo_kie in y_cookie:
    if type(y_cookie[coo_kie]) == str:
        print(X_cookie_filenames[coo_kie])
        print(coo_kie,y_cookie[coo_kie])

In [161]:
# torch.save(y_cookie,'cookie_target_dict.pt')

In [162]:
# torch.save(y_sentence,'sentence_target_dict.pt')

---------------------------------------------------------------

Load Vocab Embeddings

In [31]:
glove_path = './PretrainedWordEmb/'

In [32]:
vocab_emb = pickle.load(open(f'{glove_path}/addb.vocab_emb.glove.42B.300.pkl', 'rb'))

In [33]:
len(vocab_emb)

2188

In [34]:
pos_dict = torch.load('pos_dict.pt')

426

In [267]:
num_samples

792

In [59]:
# valid_ix = random.sample(range(num_samples),int(.20*num_samples))

In [61]:
# train_ix = [i for i in range(num_samples) if i not in valid_ix]

In [63]:
len(valid_ix)

158

In [62]:
len(train_ix)

634

In [65]:
# torch.save(valid_ix,'valid_ix.pt')

In [67]:
# torch.save(train_ix,'train_ix.pt')

In [68]:
valid_ix = torch.load('valid_ix.pt')

In [69]:
len(valid_ix)

158

In [70]:
train_ix = torch.load('train_ix.pt')

In [71]:
len(train_ix)

634

In [276]:
def get_item(ix=None):
    
    # first should be...all zeros: each index represents greater than .. but you cant have a zero score..
    def num_vectorize(t):

        v = torch.zeros([30])
        v[:t] = 1

        return v.double()
    
    ix = random.randint(1,num_samples-1) if ix is None else ix
    
    #index, filename
    if ix in X_cookie_filenames.keys():
        _file = (ix,X_cookie_filenames[ix])
        _reader = X_cookie_reader
        _targetdict = y_cookie
        
    else:
        _file = (ix,X_sent_filenames[ix])
        _reader = X_sentence_reader
        _targetdict = y_sentence
        
    file,data,targetdict = (_file,_reader,_targetdict)
    
    embeddings = []
    targets = []

    embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file[1]] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file[1]] for (tokensraw,pos,tokenstem,dependency) in utterance])]
    target = targetdict[ix]

    for tkn in embedding:

        tkn[1][tkn[2]] = 1
#     print(target)   
    return [torch.cat((tkn[0],tkn[1])) for tkn in embedding], num_vectorize(target)


In [175]:
e,t = get_item(1)

30


In [126]:
len(t)

29

In [127]:
#pad with zeros -- maybe
# end with eos tensor, tag

def get_minibatch(batchsize=1, ix=None):
    
    
    def num_vectorize(targets):
        vectors = []
        for i in targets:
            v = torch.zeros([30])
            v[:i] = 1
            vectors.append(v)
        return vectors
    
    
    minibatch_ix = random.sample(range(num_samples),batchsize) if ix is None else ix 
    
    #index, filename
    cookie_files = [(i,X_cookie_filenames[i]) for i in minibatch_ix if i in X_cookie_filenames.keys()]
    sent_files = [(i,X_sent_filenames[i] )for i in minibatch_ix if i in X_sent_filenames.keys()]
    
    embeddings = []
    targets = []

    for corpus,data,targetdict in [(cookie_files,X_cookie_reader,y_cookie),(sent_files,X_sentence_reader,y_sentence)]: 
        for file_ix,file in corpus:
#             print(file_ix, file)
#             print('Words',[tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])
#             print('Words_len',len([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance]))
            embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])]
            target = targetdict[file_ix]
            
            for tkn in embedding:

                tkn[1][tkn[2]] = 1

#                 embeddings.append(torch.cat((tkn[0],tkn[1])))
            
            embeddings.append([torch.cat((tkn[0],tkn[1])) for tkn in embedding])


#             embeddings.append(embedding)
            targets.append(target)
    print(targets)       
#     return embeddings, torch.tensor(targets), minibatch_ix
#     return embeddings, torch.tensor(targets)
    return embeddings, num_vectorize(targets)

In [217]:
e,t = get_minibatch(batchsize=num_samples)

[20, 30, 26, 29, 19, 23, 28, 26, 23, 19, 29, 30, 29, 30, 29, 20, 30, 20, 29, 30, 21, 5, 30, 30, 29, 19, 15, 13, 29, 10, 16, 15, 30, 7, 30, 30, 28, 30, 30, 20, 28, 29, 13, 30, 26, 30, 24, 25, 24, 30, 25, 30, 19, 19, 19, 28, 24, 28, 30, 17, 28, 18, 30, 30, 27, 25, 30, 19, 29, 30, 26, 17, 10, 30, 28, 17, 30, 15, 27, 30, 30, 22, 30, 29, 27, 21, 30, 19, 15, 17, 27, 16, 30, 28, 24, 26, 19, 30, 29, 26, 15, 30, 30, 30, 22, 29, 28, 22, 30, 26, 30, 30, 28, 28, 30, 17, 20, 27, 25, 30, 28, 30, 20, 23, 30, 30, 30, 29, 29, 30, 20, 17, 19, 17, 15, 30, 21, 28, 22, 28, 29, 10, 28, 22, 22, 13, 30, 15, 29, 19, 28, 20, 30, 20, 29, 30, 30, 29, 30, 29, 29, 29, 30, 19, 19, 27, 18, 28, 28, 12, 30, 30, 12, 29, 30, 3, 30, 23, 19, 30, 11, 12, 19, 30, 24, 24, 29, 19, 16, 19, 30, 28, 25, 20, 30, 25, 20, 29, 10, 30, 30, 19, 30, 28, 15, 30, 29, 8, 20, 24, 30, 30, 28, 29, 28, 23, 30, 23, 29, 30, 30, 28, 16, 30, 30, 20, 26, 13, 28, 27, 30, 30, 30, 30, 23, 28, 18, 29, 24, 22, 30, 11, 13, 21, 30, 26, 13, 28, 20, 23, 23,

In [138]:
len(np.where(t)[0])

29

In [142]:
len(np.where(torch.zeros([29]))[0])

0

In [232]:
# accuracy measurement
# return summation -- doesnt perfectly handle skipped values
# np.where(vector)[0][-1] https://stackoverflow.com/questions/38375401/neural-network-ordinal-classification-for-age
def out_to_score(vector):
    # zeros array 
    
    return vector.sum() + 1
    
    if len(np.where(vector)[0]) == 0: return 0
#     print ([i+2 for i in np.where(vector)])
    print(np.where(vector))
    candidate = (np.where(vector)[0][-1]) 
    
    # handling intermediate zeros...maybe not optimal
    if np.where(vector)[0][-2] < (candidate - 1): candidate = (np.where(vector)[0][-2] + np.where(vector)[0][-1])/2
        
    return candidate + 1
    # alternative
    return vector.sum() + 1
#     len(np.where(t)[0])

In [218]:
# batch of all samples:  longest sample length (for padding, if minibatch)
max([len(emb) for emb in e])

546

In [199]:
# all words same len of emb size 371 
set([len(emb[i]) for emb in e for i in range(len(emb))])

{371}

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

Seq2Seq -- with attention

2 layer rnn..

In [215]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

371

In [219]:
HIDDEN_SIZE = 128
# EMBEDDING_SIZE = 371
# with EOS pos
EMBEDDING_SIZE = len(vocab_emb['I']) + len(pos_dict)

In [268]:
# max([len(emb) for emb in e])

546

In [74]:
# change if aadd in EOS, pos to 547
MAX_LENGTH = 546

In [75]:
VOCAB_SIZE = len(vocab_emb)

In [101]:
len(dict)

71

In [102]:
# pos_dict['EOS'] = len(pos_dict)

In [104]:
# EOS tag
# vocab_emb['EOS'] = torch.rand([300],dtype=torch.float64)

In [253]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size=MAX_LENGTH, emb_size=EMBEDDING_SIZE, hidden_size=HIDDEN_SIZE):
        super(EncoderRNN, self).__init__()
        
        self.emb_size = emb_size
        self.hidden_size = hidden_size

        # what is the input size fo self.embedding...? and is self.embedding necessary
#         self.embedding = nn.Embedding(input_size, emb_size)
        
        
        
#         self.gru = nn.GRU(hidden_size, hidden_size)
#         self.gru = nn.GRU(input_size=self.emb_size, hidden_size=self.hidden_size,num_layers=2)
        self.gru = nn.GRU(input_size=self.emb_size,hidden_size=self.hidden_size,num_layers=2).double()
    
#         # each output node Oi of our neural network
# uses a standard sigmoid function 1
# 1+e−zi
# , without including
# the outputs from other nodes, as shown in Figure 1. Output
# node Oi
# is used to estimate the probability oi
# that a data
# point belongs to category i independently, without subjecting
# to normalization as traditional neural networks do. Thus,
# for a data point x of category k, the target vector is
# (1, , 1, .., 1, 0, 0, 0), in which the first k elements is 1 and
# others 0
# http://orca.st.usm.edu/~zwang/files/rank.pdf A Neural Network Approach to Ordinal Regression
    
        self.fc = nn.Linear(self.hidden_size*2,30).double()

    def forward(self, input, hidden):
        # confirm dims here...
#         embedded = self.embedding(input).view(1, 1, -1)
        
        embedded = torch.cat(input).view(len(input), 1, -1).double()
        
#         output = embedded
        
#         output, hidden = self.gru(self.emb_size, self.hidden_size)
#         output, hidden = self.gru(output)
        gru_out, _ = self.gru(embedded,hidden)
        
        hid_out = torch.cat((_[-2,:,:], _[-1,:,:]), dim = 1)
#         print(gru_out.size(),_.size())
    
#         lstm_out, _ = self.lstm(embedding.view(len(sentence), -1))
        out = self.fc(hid_out)
        
        # any further processing -- int? softmax...? 
#         return out
        return out, _

    def initHidden(self):
        # h_0 of shape (num_layers * num_directions, batch, hidden_size): tensor containing the initial hidden state for each element in the batch. If the LSTM is bidirectional, num_directions should be 2
        return torch.zeros(2, 1, self.hidden_size, device=device).double()


In [266]:
# The
# cost function for a data point x can be relative entropy
# or square error between the target vector and the output
# vector
# loss = nn.BCELoss()
loss_func = nn.MSELoss()

# TODO minibatch + pad. For starters, stochastic gd..

In [267]:
encoder = EncoderRNN()

In [268]:
optimizer = torch.optim.Adam(encoder.parameters(), lr=1e-2)

In [273]:
yhat

tensor([[ 4.6224e-02, -3.7343e-02, -1.4654e-01, -3.5154e-02,  1.0044e-01,
          8.3315e-02,  2.8276e-02, -1.9517e-01, -1.1097e-01,  2.0583e-02,
          7.1402e-03, -2.1361e-04, -1.5765e-01, -1.5639e-01, -1.1031e-02,
         -1.4089e-01, -4.7802e-02,  1.6864e-01, -2.2820e-01,  1.2027e-02,
         -1.7279e-01, -4.5319e-03, -1.1785e-01, -1.6160e-01, -7.9826e-02,
          2.0204e-01, -5.6356e-02,  6.1862e-02,  3.7473e-02,  8.5436e-02]],
       dtype=torch.float64, grad_fn=<AddmmBackward>)

In [275]:
y.double()

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=torch.float64)

In [ ]:
encoder.train()
_hidden = encoder.initHidden()

for epoch in range(10):
    
    step = 0
    
    # in future, mix up epochs 
    for i in train_ix:
        
        step += 1
        optimizer.zero_grad()
        
        x,y = get_item(i)

        yhat, _hidden = encoder(x,_hidden)

        loss = loss_func(yhat,y)

        loss.backward(retain_graph=True)
        
        optimizer.step()
        
        if step % 20 == 0: print(loss)

    
    
    

tensor(0.0433, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.0239, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.0200, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.0109, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.0151, dtype=torch.float64, grad_fn=<MseLossBackward>)


In [290]:
x,y = get_minibatch(batchsize=1)

In [287]:
torch.cat(x[0]).view(len(x[0]),1, -1).shape

torch.Size([55, 1, 371])

In [251]:
# need to be able to pass any/all into net
# encoder = EncoderRNN(len(x[0])).to(device)

In [293]:
_hidden.size()

torch.Size([2, 1, 128])